In [17]:
import pandas as pd
from ruamel import yaml
from simple_salesforce import Salesforce
import great_expectations as gx
from great_expectations.core.batch import RuntimeBatchRequest
import os

sf_client = Salesforce(
    username=os.getenv("SF_USERNAME"),
    password=os.getenv("SF_PASSWORD"),
    security_token=os.getenv("SF_TOKEN")
)

# Configure Context

In [18]:
context = gx.data_context.FileDataContext.create("/home/jovyan/gx_suites/poc2")

In [19]:
# datasource_config = {
#     "name": "example_datasource",
#     "class_name": "Datasource",
#     "module_name": "great_expectations.datasource",
#     "execution_engine": {
#         "module_name": "great_expectations.execution_engine",
#         "class_name": "PandasExecutionEngine",
#     },
#     "data_connectors": {
#         "default_runtime_data_connector_name": {
#             "class_name": "RuntimeDataConnector",
#             "module_name": "great_expectations.datasource.data_connector",
#             "batch_identifiers": ["OwnerId"],
#         },
#     },
# }

# context.add_datasource(**datasource_config)

# Pull data

In [20]:
resp = sf_client.query(
    """
    SELECT 
    Id,
    Agent_ID__c,
    CreatedById,
    Current_Status__c,
    LastModifiedById,
    OwnerId,
    Price__c,
    Name,
    Street_Address__c,
    Unique_Street_Adddress__c
    FROM Property__c
    """
)


In [21]:
df = pd.DataFrame.from_records(resp['records'])

In [22]:
df

,attributes,Id,Agent_ID__c,CreatedById,Current_Status__c,LastModifiedById,OwnerId,Price__c,Name,Street_Address__c,Unique_Street_Adddress__c
0,"{'type': 'Property__c', 'url': '/services/data...",a00Hp00001DLSiMIAX,None,005Hp00000h5ERFIA2,None,005Hp00000h5ERFIA2,005Hp00000h5ERFIA2,500000.0,123 Main St,None,None
1,"{'type': 'Property__c', 'url': '/services/data...",a00Hp00001DQk7KIAT,a05Hp000014GDkCIAW,005Hp00000h5ERFIA2,None,005Hp00000h5ERFIA2,005Hp00000h5ERFIA2,1000000.0,1600 Penn Ave,1600 Pennsylvania Avenue,None
2,"{'type': 'Property__c', 'url': '/services/data...",a00Hp00001DSMyZIAX,a05Hp000014GIbBIAW,005Hp00000h5ERFIA2,None,005Hp00000h5ERFIA2,005Hp00000h5ERFIA2,499000.0,123 Main St,123 Main St,None
3,"{'type': 'Property__c', 'url': '/services/data...",a00Hp00001DSP0GIAX,a05Hp000014GDkCIAW,005Hp00000h5ERFIA2,Listed,005Hp00000h5ERFIA2,005Hp00000h5ERFIA2,45000.0,Another property again,978 Keefer Rd,978 Keefer Rd
4,"{'type': 'Property__c', 'url': '/services/data...",a00Hp00001DSP06IAH,a05Hp000014GDkCIAW,005Hp00000h5ERFIA2,Listed,005Hp00000h5ERFIA2,005Hp00000h5ERFIA2,50000.0,Another property,40 Stefanyk Rd,40 Stefanyk Rd


In [34]:
# checkpoint_config = {
#     "name": "my_checkpoint",
#     "config_version": 1,
#     "class_name": "SimpleCheckpoint",
#     "validations": [
#         {
#             "batch_request": {
#                 "datasource_name": "example_datasource",
#                 "data_connector_name": "default_runtime_data_connector_name",
#                 "data_asset_name": "Property__c",
#             },
#             "expectation_suite_name": "Property__c",
#         }
#     ],
# }
# checkpoint = context.add_checkpoint(**checkpoint_config)

In [30]:
results = context.run_checkpoint(
    checkpoint_name="my_checkpoint",
    batch_request={
        "runtime_parameters": {"batch_data": df},
        "batch_identifiers": {
            "run_id": "run1"
        },
    },
    run_name="run1"
)

Calculating Metrics:   0%|          | 0/75 [00:00<?, ?it/s]

In [31]:
context.build_data_docs()

{'local_site': 'file:///home/jovyan/gx_suites/poc2/gx/uncommitted/data_docs/local_site/index.html'}